In [ ]:
import xarray as xr
import healpy as hp
import easygems.healpix as egh
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs

In [ ]:
# read ICON model output
zoom = '5'
file = '/g/data/qx55/germany_node/d3hp003.zarr/PT1H_point_z' + zoom + '.zarr'

ds2d = xr.open_zarr(file)
ds2d

pr_icon = ds2d['pr'] #precipitation

this_time = '2020-01-26T00:00:00'
egh.healpix_show(pr_icon.sel(time = this_time).squeeze(), cmap = 'viridis')
plt.title('ICON total precipitation ' + this_time + ' HEALPix Zoom '+ zoom)


In [ ]:
pr_icon_month = pr_icon.groupby("time.month").mean() # Calculate monthly mean.

this_month = 2
egh.healpix_show(pr_icon_month.sel(month = this_month).squeeze(), cmap = 'viridis')
plt.title('icon monthly total precipitation for February - HEALPix Zoom '+ zoom)

In [ ]:
# read imerg data
file_IMERG = '/g/data/ia39/aus-ref-clim-data-nci/frogs/data/1DD_V1/IMERG_V07B_FC/IMERG_V07B_FC.1DD.*.nc'

pr_imerg = xr.open_mfdataset(file_IMERG, combine = 'nested', concat_dim = 'time')
pr_imerg_monthly = pr_imerg['rain'].groupby('time.month').mean() # Calculate monthly mean.


In [ ]:
# plot for IMERG monthly mean
projection = ccrs.PlateCarree(central_longitude = 120)

fig, ax = plt.subplots(subplot_kw = {"projection": projection})
pr_imerg_monthly.sel(month = 2).plot(cmap = 'viridis', transform = ccrs.PlateCarree())
ax.coastlines()
ax.set_title('IMERG total precipitation for February 2020')

In [ ]:
#regridding for maritime continent
pr_imerg_monthly_mari = pr_imerg_monthly.sel(longitude = slice(100, 149), latitude = slice(-10, 10))


In [ ]:
def get_nn_lon_lat_index(nside, lons, lats):
    """
    nside: integer, power of 2. The return of hp.get_nside()
    lons: uniques values of longitudes
    lats: uniques values of latitudes
    returns: array with the HEALPix cells that are closest to the lon/lat grid
    """
    lons2, lats2 = np.meshgrid(lons, lats)
    return xr.DataArray(
        hp.ang2pix(nside, lons2, lats2, nest = True, lonlat = True),
        coords=[("lat", lats), ("lon", lons)],
    )

In [ ]:
# Find the HEALPix pixels that are closest to the imerg grid

# longitudes and latitudes for the imerg grid
lon = pr_imerg_monthly_mari['longitude'].values
lat = pr_imerg_monthly_mari['latitude'].values

# nside for icon simulation, it should be equal to 2**zoom
this_nside = hp.get_nside(pr_imerg_monthly)

cells = get_nn_lon_lat_index(this_nside, lon, lat) 

cells

In [ ]:
# Plot icon simulation regrided to lon/lat from imerg
fig, ax = plt.subplots(subplot_kw = {"projection": projection})
pr_icon_month.isel(cell = cells).sel(month = 2).plot(cmap = 'viridis', transform = ccrs.PlateCarree())
ax.coastlines()
ax.set_title('ICON total precipitation for February 2020')

In [ ]:
# calculate the difference

pr_icon_month_regrided = pr_icon_month.isel(cell = cells) # regriding
pr_icon_month_regrided = pr_icon_month_regrided.rename({'lon': 'longitude', 'lat': 'latitude'}) # we need to change the names to match ERA5 data

In [ ]:
difference = pr_icon_month_regrided.sel(month = 2) - pr_erapr_icon_month_regrided.sel(month = 2) # difference ICON - ERA5

fig, ax = plt.subplots(subplot_kw = {"projection": projection})
difference.plot(ax = ax, cmap = 'RdBu', transform = ccrs.PlateCarree())
ax.coastlines()
ax.set_title('Difference icon - imerg, total precipitation 02/2020')